# NOYES TEAM!

In [4]:
#import numpy and data science package
import matplotlib
matplotlib.use('Agg')
from datascience import Table
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import numpy as np
plt.style.use('fivethirtyeight')

In [ ]:
#intro?

In [35]:
#describing the table
covid = Table.read_table('DOCCS DATA - Data Fest Table.csv')
covid

Date,Prison Pos. Cases,Total Cases (from total population of NY),Prison Pos. % Change,Total Pos. Cases % Change,Prison daily,Total NY daily,Prison Rate of Inf.,NY Rate of Inf.,Difference of Rates of Infection
19-Apr,211,247815,nan,nan,nan,nan,nan,nan,nan
20-Apr,219,253060,3.79%,2.12%,8,5245,nan,nan,nan
21-Apr,239,258361,9.13%,2.09%,20,5301,nan,nan,nan
22-Apr,263,263292,10.04%,1.91%,24,4931,nan,nan,nan
23-Apr,269,263460,2.28%,0.06%,6,168,nan,nan,nan
24-Apr,287,271590,6.69%,3.09%,18,8130,nan,nan,nan
25-Apr,296,282143,3.14%,3.89%,9,10553,nan,nan,nan
26-Apr,310,288045,4.73%,2.09%,14,5902,nan,nan,nan
27-Apr,330,291996,6.45%,1.37%,20,3951,nan,nan,nan
28-Apr,343,295106,3.94%,1.07%,13,3110,nan,nan,nan


In [ ]:
#take the log of the number of cases in prison

In [44]:
covid['Log Prison Pos. Cases'] = np.log(covid['Prison Pos. Cases'])
covid

Date,Prison Pos. Cases,Total Cases (from total population of NY),Prison Pos. % Change,Total Pos. Cases % Change,Prison daily,Total NY daily,Prison Rate of Inf.,NY Rate of Inf.,Difference of Rates of Infection,Log Prison Pos. Cases,Log Total Cases
19-Apr,5.35186,247815,nan,nan,nan,nan,nan,nan,nan,1.67744,12.4204
20-Apr,5.38907,253060,3.79%,2.12%,8,5245,nan,nan,nan,1.68437,12.4414
21-Apr,5.47646,258361,9.13%,2.09%,20,5301,nan,nan,nan,1.70046,12.4621
22-Apr,5.57215,263292,10.04%,1.91%,24,4931,nan,nan,nan,1.71778,12.481
23-Apr,5.59471,263460,2.28%,0.06%,6,168,nan,nan,nan,1.72182,12.4816
24-Apr,5.65948,271590,6.69%,3.09%,18,8130,nan,nan,nan,1.73333,12.512
25-Apr,5.69036,282143,3.14%,3.89%,9,10553,nan,nan,nan,1.73877,12.5501
26-Apr,5.73657,288045,4.73%,2.09%,14,5902,nan,nan,nan,1.74686,12.5709
27-Apr,5.79909,291996,6.45%,1.37%,20,3951,nan,nan,nan,1.7577,12.5845
28-Apr,5.83773,295106,3.94%,1.07%,13,3110,nan,nan,nan,1.76434,12.5951


In [ ]:
#take the log of the number of cases in New York

In [45]:
covid['Log Total Cases'] = np.log(covid['Total Cases (from total population of NY)']-covid['Prison Pos. Cases'])
covid

Date,Prison Pos. Cases,Total Cases (from total population of NY),Prison Pos. % Change,Total Pos. Cases % Change,Prison daily,Total NY daily,Prison Rate of Inf.,NY Rate of Inf.,Difference of Rates of Infection,Log Prison Pos. Cases,Log Total Cases
19-Apr,5.35186,247815,nan,nan,nan,nan,nan,nan,nan,1.67744,12.4204
20-Apr,5.38907,253060,3.79%,2.12%,8,5245,nan,nan,nan,1.68437,12.4414
21-Apr,5.47646,258361,9.13%,2.09%,20,5301,nan,nan,nan,1.70046,12.4621
22-Apr,5.57215,263292,10.04%,1.91%,24,4931,nan,nan,nan,1.71778,12.481
23-Apr,5.59471,263460,2.28%,0.06%,6,168,nan,nan,nan,1.72182,12.4816
24-Apr,5.65948,271590,6.69%,3.09%,18,8130,nan,nan,nan,1.73333,12.512
25-Apr,5.69036,282143,3.14%,3.89%,9,10553,nan,nan,nan,1.73877,12.5501
26-Apr,5.73657,288045,4.73%,2.09%,14,5902,nan,nan,nan,1.74686,12.5709
27-Apr,5.79909,291996,6.45%,1.37%,20,3951,nan,nan,nan,1.7577,12.5845
28-Apr,5.83773,295106,3.94%,1.07%,13,3110,nan,nan,nan,1.76434,12.5951
